# Testing notebook

In [2]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

Data generator for the study of interrelationships

In [3]:
def inter_data_gen(grop_size = 200, grops_count = 3, relation_param = 1.2):
    # lets creaate a sample wich contains columns for every situation
    Y = np.concatenate([np.zeros(grop_size) + i for i in range(grops_count)])
    related_col = np.concatenate(
        [np.random.normal(i*relation_param, 0.5, grop_size) for i in range(grops_count)]
    )
    non_related_col = np.concatenate(
        [np.random.normal(0, 0.5, grop_size) for i in range(grops_count)]
    )
    # get <grops_count>-order quantiles
    related_col_quants = np.array(
        [np.quantile(related_col,(i+1)/grops_count) for i in range(grops_count)]
    )
    non_related_col_quants = np.array(
        [np.quantile(non_related_col,(i+1)/grops_count) for i in range(grops_count)]
    )
    test_frame = pd.DataFrame({
        "Y" : Y,
        "related_col" : related_col,
        "non_related_col" : non_related_col,
        "related_col_cat" : \
            map(lambda x: sum(x > related_col_quants), related_col),
        "non_related_col_cat":\
            map(lambda x: sum(x > non_related_col_quants), non_related_col)
    })
    
    test_frame["Y_names"] =\
    test_frame['Y'].replace({
        level : "name " + str(level) \
        for level in test_frame['Y'].unique()
    })
    
    return test_frame

temp_frame = inter_data_gen()
temp_frame

,Y,related_col,non_related_col,related_col_cat,non_related_col_cat,Y_names
0,0.0,-0.604477,-0.116845,0,1,name 0.0
1,0.0,0.165630,-0.631073,0,0,name 0.0
2,0.0,0.473055,-0.007338,0,1,name 0.0
3,0.0,-0.130910,-0.926020,0,0,name 0.0
4,0.0,0.097606,-0.534041,0,0,name 0.0
...,...,...,...,...,...,...
595,2.0,2.343234,-0.309851,2,0,name 2.0
596,2.0,2.057881,-0.312358,2,0,name 2.0
597,2.0,2.522542,0.233154,2,2,name 2.0
598,2.0,2.231606,-0.257785,2,1,name 2.0


# Computions funcitons

In [4]:
from computions import *

## get_describe_nominal function test

Basic situation

In [5]:
get_describe_nominal(temp_frame["related_col_cat"], temp_frame.Y)

,count,0.0,0.0%,1.0,1.0%,2.0,2.0%
related_col_cat,,,,,,,
0,200,173.0,86.5,27,13.5,0.0,0.0
1,200,27.0,13.5,155,77.5,18.0,9.0
2,200,0.0,0.0,18,9.0,182.0,91.0


## Stats computing test

Testig settings

In [6]:
def stats_error_mesage(error_type):
    print("ERROR:")
    print(error_type)
    print("may be try again random is random")

KS_conv_level = 0.05
second_kind_error_count = 0
first_kind_error_count = 0
test_count = 200

Stats numeric

In [7]:
for i in range(test_count):
    data_for_testing = inter_data_gen(relation_param = 1.2)
    
    related_stats = get_stats_numeric(
        data_for_testing['related_col'], data_for_testing['Y']
    )
    non_related_stats = get_stats_numeric(
        data_for_testing['non_related_col'], data_for_testing['Y']
    )
    
    if related_stats[2]['AUC'] < non_related_stats[2]['AUC']:  
        stats_error_mesage(
            'AUC of related sample is smaller than AUC of independent!'
        )
        break
    if related_stats[0]['AUC'] > 0.5:
        stats_error_mesage(
            'AUC for factor with smaller on average levels must be lower than 0.5'
        )
        break
    if related_stats[2]['KS'] < non_related_stats[2]['KS']:
        stats_error_mesage(
            'KS of related sample is smaller than KS of independent!'
        )
    
    if related_stats[0]['KS_p_val'] > KS_conv_level:
        first_kind_error_count += 1
    if non_related_stats[0]['KS_p_val'] < KS_conv_level:
        second_kind_error_count += 1
        
print("KS test results")
print("first kind error")
print(
    "count " + str(first_kind_error_count) + "; share " +\
    str(np.round(first_kind_error_count*100/test_count, 3)) + "%"
)
print("second kind error")
print(
    "count " + str(second_kind_error_count) + "; share " +\
    str(np.round(second_kind_error_count*100/test_count, 3)) + "%"
)

KS test results
first kind error
count 0; share 0.0%
second kind error
count 13; share 6.5%


Stats nominal

In [8]:
for i in range(test_count):
    data_for_testing = inter_data_gen(relation_param = 1.2)
    
    related_stats = get_stats_numeric(
        data_for_testing['related_col_cat'], data_for_testing['Y']
    )
    non_related_stats = get_stats_numeric(
        data_for_testing['non_related_col_cat'], data_for_testing['Y']
    )
    
    if related_stats[2]['AUC'] < non_related_stats[2]['AUC']:  
        stats_error_mesage(
            'AUC of related sample is smaller than AUC of independent!'
        )
        break
    if related_stats[0]['AUC'] > 0.5:
        stats_error_mesage(
            'AUC for factor with smaller on average levels must be lower than 0.5'
        )
        break
    if related_stats[2]['KS'] < non_related_stats[2]['KS']:
        stats_error_mesage(
            'KS of related sample is smaller than KS of independent!'
        )
        break
        
    if related_stats[0]['KS_p_val'] > KS_conv_level:
        first_kind_error_count += 1
    if non_related_stats[0]['KS_p_val'] < KS_conv_level:
        second_kind_error_count += 1
        
print("KS test results")
print("first kind error")
print(
    "count " + str(first_kind_error_count) + "; share " +\
    str(np.round(first_kind_error_count*100/test_count, 3)) + "%"
)
print("second kind error")
print(
    "count " + str(second_kind_error_count) + "; share " +\
    str(np.round(second_kind_error_count*100/test_count, 3)) + "%"
)

KS test results
first kind error
count 0; share 0.0%
second kind error
count 13; share 6.5%


Does it work with description table

In [9]:
#Y_strs = ["name" + str(i) for i in range(3)]


dn_tab = get_describe_nominal(
    temp_frame["related_col_cat"], 
    temp_frame["Y_names"]
)

with_table = get_stats_nominal(
    temp_frame["related_col_cat"], 
    temp_frame["Y_names"], dn_tab
)
with_table

{'name 0.0': {'AUC': 0.9325, 'KS': 0.7975, 'KS_p_val': 2.3581393559109397e-91},
 'name 1.0': {'AUC': 0.8425, 'KS': 0.6625, 'KS_p_val': 6.731549661221255e-58},
 'name 2.0': {'AUC': 0.9550000000000001,
  'KS': 0.865,
  'KS_p_val': 6.553529640023942e-116}}

Getting full info about stats - no difference nominal or numeric

In [10]:
data_for_testing = inter_data_gen(relation_param = 1.2)
print("result for numeric column")
print(get_full_stats(
    data_for_testing["related_col"], data_for_testing.Y, "numeric"
))
print("result for nominal column")
print(get_full_stats(
    data_for_testing["related_col_cat"], data_for_testing.Y, "nominal"
))

result for numeric column
{0.0: {'AUC': 0.9674625, 'KS': 0.7975, 'KS_p_val': 2.3581393559109397e-91, 'rel_type': -1, 'GINI': 0.934925, 'Empty': 0}, 1.0: {'AUC': 0.505525, 'KS': 0.36250000000000004, 'KS_p_val': 3.6419462233367737e-16, 'rel_type': -1, 'GINI': 0.011050000000000004, 'Empty': 0}, 2.0: {'AUC': 0.9729875000000001, 'KS': 0.8150000000000001, 'KS_p_val': 5.753569336943113e-97, 'rel_type': 1, 'GINI': 0.9459750000000002, 'Empty': 0}}
result for nominal column
{0.0: {'AUC': 0.925, 'KS': 0.775, 'KS_p_val': 1.0826093372722408e-84, 'rel_type': 1, 'GINI': 0.8500000000000001, 'Empty': 0}, 1.0: {'AUC': 0.7949999999999999, 'KS': 0.58, 'KS_p_val': 5.466245179415354e-43, 'rel_type': 1, 'GINI': 0.5899999999999999, 'Empty': 0}, 2.0: {'AUC': 0.935, 'KS': 0.8049999999999999, 'KS_p_val': 1.0418503176643625e-93, 'rel_type': 1, 'GINI': 0.8700000000000001, 'Empty': 0}}


**And finally all computions**

In [11]:
data_for_testing = inter_data_gen(relation_param = 1.2)

print('describe_table with no nas')
print(get_all_comuptions(
    data_for_testing['related_col'], 
    data_for_testing.Y
)['describe_table'])

data_for_testing.loc[data_for_testing.sample(20).index] = np.NaN
print('describe_table with nas')
print(get_all_comuptions(
    data_for_testing['related_col'], 
    data_for_testing.Y
))

describe_table with no nas
           Value:  Part %:
count  600.000000      NaN
mean     1.188007      NaN
std      1.142691      NaN
min     -1.435743      NaN
25%      0.294360      NaN
50%      1.185268      NaN
75%      2.106443      NaN
max      4.057332      NaN
Empty    0.000000      0.0
describe_table with nas
{'name': 'related_col', 'emptys_count': 20, 'emptys_part': 0.03333333333333333, 'predictor_type': 'numeric', 'describe_table':            Value:   Part %:
count  580.000000       NaN
mean     1.196314       NaN
std      1.139676       NaN
min     -1.435743       NaN
25%      0.314268       NaN
50%      1.189922       NaN
75%      2.106443       NaN
max      4.057332       NaN
Empty   20.000000  3.333333, 'stats_result': {0.0: {'AUC': 0.9810862328178694, 'KS': 0.8602341065292096, 'KS_p_val': 1.3413041963616654e-109, 'rel_type': -1, 'GINI': 0.9621724656357389, 'Empty': 20}, 1.0: {'AUC': 0.5056543456543456, 'KS': 0.37229437229437234, 'KS_p_val': 1.4788265366394843e-16, 'rel

In [12]:
data_for_testing = inter_data_gen(relation_param = 1.2)

stats_info_to_DataFrame(
    get_all_comuptions(
        data_for_testing['non_related_col'], 
        data_for_testing.Y
    )['stats_result']
)

0.0                                              1.0                  \
        AUC    KS KS_p_val rel_type     GINI Empty       AUC    KS  KS_p_val   
0  0.532775  0.09  0.21804        1  0.06555     0  0.518863  0.06  0.701569   

                                 2.0                                            
  rel_type      GINI Empty       AUC     KS  KS_p_val rel_type      GINI Empty  
0       -1  0.037725     0  0.513912  0.075  0.422296       -1  0.027825     0

Getting indicators from computions uotput

In [13]:
def get_predictor_row(column_data):

    result = stats_info_to_DataFrame(column_data['stats_result'], column_data['name'])
    result['Emptys'] = column_data['emptys_count']
    result['Emptys part'] = column_data['emptys_part']
    return result

data_for_testing = inter_data_gen(relation_param = 1.2)

comp_result =  get_all_comuptions(
    data_for_testing['non_related_col'], 
    data_for_testing.Y
)

print('full numeric data')
get_predictor_row(comp_result)

full numeric data


0.0                                               1.0  \
                      AUC     KS  KS_p_val rel_type      GINI Empty     AUC   
non_related_col  0.528863  0.085  0.275855        1  0.057725     0  0.5267   

                                                               2.0         \
                     KS  KS_p_val rel_type    GINI Empty       AUC     KS   
non_related_col  0.0875  0.245688       -1  0.0534     0  0.502163  0.055   

                                                  Emptys Emptys part  
                KS_p_val rel_type      GINI Empty                     
non_related_col  0.79504       -1  0.004325     0      0         0.0

# Writing to excel

Testing data

In [14]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

Adding a info about different predictors

In [15]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

Let's show final table

In [16]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.get_predictors_data()
my_cpp.result_DF

1.0                      \
                                                    AUC        KS  KS_p_val   
numeric_variable                                0.52574  0.069639  0.175453   
object_variable                                0.828972  0.502429       0.0   
column with soo000oo000oo000oo000oo long name  0.518403  0.070256  0.168087   

                                                                        \
                                              rel_type      GINI Empty   
numeric_variable                                     1   0.05148     0   
object_variable                                      1  0.657945     0   
column with soo000oo000oo000oo000oo long name        1  0.036806     0   

                                                    0.0                      \
                                                    AUC        KS  KS_p_val   
numeric_variable                               0.690462  0.293914       0.0   
object_variable                                0.942945  0.720029       0.0   
column with soo000oo000oo000oo000oo long name  0.513861  0.051165  0.628651   

                                                                        \
                                              rel_type      GINI Empty   
numeric_variable                                    -1  0.380923     0   
object_variable                                      1  0.885891     0   
column with soo000oo000oo000oo000oo long name       -1  0.027722     0   

                                                    2.0                      \
                                                    AUC        KS  KS_p_val   
numeric_variable                               0.769316  0.422573       0.0   
object_variable                                0.914622  0.689642       0.0   
column with soo000oo000oo000oo000oo long name   0.51341   0.09839  0.172827   

                                                                       Empty  \
                                              rel_type      GINI Empty         
numeric_variable                                     1  0.538633     0     0   
object_variable                                      1  0.829245     0     0   
column with soo000oo000oo000oo000oo long name       -1   0.02682     0     0   

                                              Empty part  
                                                          
numeric_variable                                     0.0  
object_variable                                      0.0  
column with soo000oo000oo000oo000oo long name        0.0

Writig a double header table

In [17]:
xl_writer = pd.ExcelWriter("test_result/double_header_saver.xlsx",engine='xlsxwriter')
ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws      

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

print_double_column_header(  my_cpp, ws, 
                            my_cpp.result_DF.loc[['numeric_variable'],:],
                            "A1", 'Hello World', 
                            xl_writer.book.add_format(my_cpp.default_header_format))

xl_writer.close()

Writing info about some predictor

In [18]:
xl_writer = pd.ExcelWriter("test_result/writing_a_value.xlsx",engine='xlsxwriter')

ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws                             

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

default_predictor_printer(my_cpp, ws, my_cpp.get_predictors_data()['numeric_variable'], 
{'header_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'desc_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'class_ab_format':xl_writer.book.add_format(my_cpp.default_header_format)})

xl_writer.close()

In [19]:
xl_writer = pd.ExcelWriter("test_result/result_test.xlsx",engine='xlsxwriter')
my_cpp.write_to_book(xl_writer)
xl_writer.close()